# Retrieve Knowledge from an External Database

Again, we retrieve the data but this time we pre-defined its structure and stored it in an exteranl database.

In [ ]:
! pip install openai
! pip install pandas
! pip install duckdb

In [108]:
# Import necessary libraries 

from openai import OpenAI
from typing import Dict, Any, List

from api_utils import load_api_params

import pandas as pd
import duckdb

In [109]:
# Load API parameters and initialize client

SECRETS_PATH = ".secrets.toml"

API_CALL_PARAMS = load_api_params(SECRETS_PATH)
client = OpenAI(
    base_url = API_CALL_PARAMS['API_URL'],
    api_key = API_CALL_PARAMS['API_KEY']
)

In [111]:
def generate_completion(model: str, messages: List[Dict[str, str]]) -> str:
    """Generate LLM output"""
    response = client.chat.completions.create(
        model=model, 
        messages=messages
    )
    return response.choices[0].message.content

def get_knowledge(filename):
    """Query database for explanation"""
    try:
        # connect to the data source
        data_frame = pd.read_csv('heart_attack.csv')
        db_connection = duckdb.connect(database=':memory:')
        db_connection.register('wonderland_data', data_frame)

        # build an SQL query 
        query = """
        SELECT * 
        FROM wonderland_data 
        WHERE Description = 'Explanation'
        """

        results = db_connection.execute(query).fetchall()
        print (results)
        return results

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

In [112]:
KNOWDLEGE = get_knowledge('heart_attack.csv')

[(6, '"A Heart Attack," mumbled the Duchess, who had appeared beside Alice. "That\'s what they call it when the Heart kingdom launches an offensive."', 'Alice, Duchess', 'Unknown', 1, 23, 'Explanation')]


In [113]:
KNOWDLEGE = str(KNOWDLEGE)

SYSTEM_PROMPT = f"""Answer all user questions to the best of your ability. Use the following text for reference:

{KNOWDLEGE}
"""

print(SYSTEM_PROMPT)


Answer all user questions to the best of your ability. Use the following text for reference:

[(6, '"A Heart Attack," mumbled the Duchess, who had appeared beside Alice. "That\'s what they call it when the Heart kingdom launches an offensive."', 'Alice, Duchess', 'Unknown', 1, 23, 'Explanation')]



In [114]:
USER_PROMPT = """Can you please tell me what is a heart attack."""

In [115]:
messages = [
    {"role": "system", "content": f"""{SYSTEM_PROMPT}"""},
    {"role": "user", "content":f"""{USER_PROMPT}"""}
]
try:
    model = API_CALL_PARAMS['MODEL']
    LLM_output = generate_completion(model, messages)
except Exception as e:
    raise Exception(f"Error generating completion: {e}")

print(LLM_output)

According to the Duchess, a "heart attack" is what they call it when the Heart kingdom launches an offensive.
